####  Training & Validation File for ANN with physics encoded structure (see input shape = 5) & MSE loss Function

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
# from tensorflow import keras
import tensorflow as tf

# %load_ext tensorboard
# import tensorboard
import os


In [2]:
from datetime import datetime
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

####  Data Paths

In [4]:
BASE_DIR_PATH = '/Users/ge72vep/Desktop/thesis/' 
DATA_PATH = 'Data/Model_8/'   
MODEL_PATH = 'models/model_8_MSE_1_secs_batch1.h5' ######## Check the data path ######## 
IMAGES_PATH = 'results/images/'    
VIDEOS_PATH = 'results/videos/'
EXP_NAME = 'MSE_exp_1sec_M8_batch1' ######## Check file name ########
VIDEO_NAME = 'MSE_1sec_M8_batch1'  ######## Check file name ########

In [5]:
PATH_TO_DATA = os.path.join(BASE_DIR_PATH, DATA_PATH)
save_model_path = os.path.join(BASE_DIR_PATH, MODEL_PATH)
save_images_path = os.path.join(BASE_DIR_PATH, IMAGES_PATH, EXP_NAME)
save_video_path = os.path.join(BASE_DIR_PATH, VIDEOS_PATH, VIDEO_NAME)
save_results_path = os.path.join(BASE_DIR_PATH, 'models', EXP_NAME+'.csv')


#### Loading dataset



In [6]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'))
val_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'val.csv'))

In [8]:
train_df = train_df.sort_values(['identifier','x','time']).reset_index(drop=True)
val_df = val_df.sort_values(['identifier','x','time']).reset_index(drop=True)

In [9]:
X_train = np.array(train_df[['x','time', 'q', 'friction_coeff', 'slope']].values.tolist())
X_val = np.array(val_df[['x','time', 'q', 'friction_coeff', 'slope']].values.tolist())

In [10]:
Y_train = np.array(train_df[['u','h']].values.tolist())
Y_val = np.array(val_df[['u','h']].values.tolist())

#### Params

In [11]:
epochs= 15

#### Model  with grid search

In [12]:
best_val_loss = np.inf
best_model = -1 

In [13]:
results = pd.DataFrame(columns=['n1','n2','n3', 'epochs', 'reg',
                               'val_r2', 'val_nse', 'val_mse', 'val_mae', 'val_mape'])
layer_1_neurons = np.arange(5,6,10)   ######## Check the number of neurons ########
layer_2_neurons = np.arange(5,26,10)   ######## Check the number of neurons ########
layer_3_neurons = np.arange(5,26,10)   ######## Check the number of neurons ########
reg_consts = [0]

#### Evaluation Metric

In [14]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

In [15]:
def NSE(y_true, y_pred):

    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [17]:
for reg in reg_consts:
    for n1 in layer_1_neurons:
        for n2 in layer_2_neurons:
            for n3 in layer_3_neurons:
                print(n1,n2,n3,reg)
                K.clear_session()
                model = Sequential()
                #model.add(Dense(n1, activation = 'relu', input_shape = (1,)))
                model.add(Dense(n1, activation = 'relu', kernel_regularizer=l2(reg),input_shape = (5,)))
                model.add(Dense(n2, activation = 'relu', kernel_regularizer=l2(reg)))
                model.add(Dense(n3, activation = 'relu', kernel_regularizer=l2(reg)))

                model.add(Dense(2))
                grads_u = K.gradients(model.output[:,0], model.input)[0]
                grads_h = K.gradients(model.output[:,1], model.input)[0]


                du_dx, du_dt, dh_dx = grads_u[:,0],grads_u[:,1],grads_h[:,0]
                calc_grads_inputs = K.stack((du_dx, du_dt, dh_dx, model.input[:,3],model.input[:,4]), axis=1)
                # model.summary()
                #Compile the model
                model.compile(optimizer = 'adam', loss = [mse], metrics=['mape', 'mae', 'mse',NSE, r_square])
                #fit the model
                early_stopping_monitor = EarlyStopping(patience = 2, verbose=False)
                history = model.fit(X_train,Y_train, epochs=epochs, batch_size=128, validation_data=(X_val,Y_val), callbacks=[early_stopping_monitor])

                # Saving results
                val_loss = history.history['val_loss'][-1]
                val_mae = history.history['val_mae'][-1]
                val_mse = history.history['val_mse'][-1]
                val_mape = history.history['val_mape'][-1]
                val_nse = history.history['val_NSE'][-1]
                val_r_square = history.history['val_r_square'][-1]

                results = results.append({'n1':n1,'n2':n2,'n3':n3, 'epochs':len(history.history['val_loss']),
                              'reg':reg,'val_r2':val_r_square, 'val_nse':val_nse, 'val_mse':val_mse, 'val_loss':val_loss,
                                        'val_mae':val_mae, 'val_mape':val_mape}, ignore_index=True)
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_model = model
                    best_n1 = n1
                    best_n2 = n2
                    best_n3 = n3
                    best_reg = reg
                    best_history = history
                    model.save(save_model_path)
                    results.to_csv(save_results_path)
                    
results.to_csv(save_results_path)

5 5 5 0
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 8877500 samples, validate on 2885500 samples
Epoch 1/15
8877500/8877500 [==============================] - 322s 36us/sample - loss: 33.3807 - mape: 58.2493 - mae: 0.6691 - mse: 16.6719 - NSE: -11.7324 - r_square: 0.6244 - val_loss: 0.6277 - val_mape: 32.5121 - val_mae: 0.4334 - val_mse: 0.2973 - val_NSE: 0.7074 - val_r_square: 0.7236
Epoch 2/15
8877500/8877500 [==============================] - 323s 36us/sample - loss: 0.7673 - mape: 37.0760 - mae: 0.4555 - mse: 0.3712 - NSE: 0.7184 - r_square: 0.7229 - val_loss: 0.5960 - val_mape: 34.2411 - val_mae: 0.4333 - val_mse: 0.2813 - val_NSE: 0.7231 - val_r_square: 0.7310
Epoch 3/15
8877500/8877500 [==============================] - 323s 36us/sample - loss: 0.7547 - mape: 36.3139 - mae: 0.4500 - mse: 0.3649 - NSE: 0.7231 - r_square: 0.7275 - val_loss: 0.5890 - val_mape: 33.8146 - val_mae: 0.4272 - val_mse: 0.2779 - val_NSE: 0.7265 - val_r_square:

8877500/8877500 [==============================] - 386s 44us/sample - loss: 0.1912 - mape: 16.4130 - mae: 0.2088 - mse: 0.0831 - NSE: 0.9367 - r_square: 0.9402 - val_loss: 0.7364 - val_mape: 32.4027 - val_mae: 0.4474 - val_mse: 0.3515 - val_NSE: 0.6537 - val_r_square: 0.6649
Epoch 4/15
8877500/8877500 [==============================] - 386s 43us/sample - loss: 0.1571 - mape: 14.3214 - mae: 0.1823 - mse: 0.0660 - NSE: 0.9498 - r_square: 0.9525 - val_loss: 0.7282 - val_mape: 33.4354 - val_mae: 0.4601 - val_mse: 0.3474 - val_NSE: 0.6581 - val_r_square: 0.6706
5 25 25 0
Train on 8877500 samples, validate on 2885500 samples
Epoch 1/15
8877500/8877500 [==============================] - 392s 44us/sample - loss: 3.6320 - mape: 37.2433 - mae: 0.4584 - mse: 1.7922 - NSE: -0.3232 - r_square: 0.7531 - val_loss: 0.5126 - val_mape: 22.4653 - val_mae: 0.3538 - val_mse: 0.2395 - val_NSE: 0.7647 - val_r_square: 0.8009
Epoch 2/15
8877500/8877500 [==============================] - 388s 44us/sample - loss

#### Model Summary

In [ ]:
best_model.summary()

#### Loss vs Epcohs Visualization 

In [ ]:
train_loss = best_history.history['loss']
val_loss = best_history.history['val_loss']    
plt.style.use('seaborn-whitegrid')
ax = plt.axes()
plt.plot(np.arange(len(train_loss)),train_loss, label ='train loss')
plt.plot(np.arange(len(val_loss)),val_loss, label ='validation loss')
plt.legend()
# plt.xticks(test['Time'].tolist()[::30], rotation = 45)
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Loss vs Epochs')
plt.show()
plt.savefig('/Users/ge72vep/Desktop/thesis/results/physics_loss_15_5_15.png', dpi=800) ######## Check the data path ######## 